In [5]:
import numpy as np
import skimage
import utils
import pathlib

In [6]:
def otsu_thresholding(im: np.ndarray) -> int:
    """
    Computes the Otsu threshold for a given grayscale image.
    
    Args:
        im (np.ndarray): A grayscale image in the range [0, 255] (dtype=np.uint8)
        
    Returns:
        int: The computed threshold value
    """
    assert im.dtype == np.uint8
    
    # Step 1: Compute normalized histogram
    hist = np.bincount(im.ravel(), minlength=256) / im.size
    
    # Step 2: Compute cumulative sums
    cumulative_sum = np.cumsum(hist)
    
    # Step 3: Compute cumulative means
    cumulative_mean = np.cumsum(np.arange(256) * hist)
    
    # Step 4: Compute global mean
    global_mean = cumulative_mean[-1]
    
    # Step 5: Compute between-class variance for each threshold k
    between_class_variance = (global_mean * cumulative_sum - cumulative_mean)**2 / (cumulative_sum * (1 - cumulative_sum))
    
    # Handle division by zero for any zero entries in cumulative_sum
    between_class_variance = np.nan_to_num(between_class_variance)
    
    # Step 6: Find the threshold k* that maximizes between-class variance
    optimal_threshold = np.argmax(between_class_variance)
    
    return optimal_threshold


In [7]:
if __name__ == "__main__":
    # DO NOT CHANGE
    impaths_to_segment = [
        pathlib.Path("thumbprint.png"),
        pathlib.Path("rice-shaded.png")
    ]
    for impath in impaths_to_segment:
        im = utils.read_image(impath)
        threshold = otsu_thresholding(im)
        print("Found optimal threshold:", threshold)

        # Segment the image by threshold
        segmented_image = (im >= threshold)
        assert im.shape == segmented_image.shape, "Expected image shape ({}) to be same as thresholded image shape ({})".format(
                im.shape, segmented_image.shape)
        assert segmented_image.dtype == bool, "Expected thresholded image dtype to be bool. Was: {}".format(
                segmented_image.dtype)

        segmented_image = utils.to_uint8(segmented_image)

        save_path = "{}-segmented.png".format(impath.stem)
        utils.save_im(save_path, segmented_image)

        


Reading image: images\thumbprint.png
Found optimal threshold: 153
Saving image to: image_processed\thumbprint-segmented.png
Reading image: images\rice-shaded.png
Found optimal threshold: 134
Saving image to: image_processed\rice-shaded-segmented.png


C:\Users\Bex\AppData\Local\Temp\ipykernel_29952\2520906564.py:26: RuntimeWarning: invalid value encountered in divide
  between_class_variance = (global_mean * cumulative_sum - cumulative_mean)**2 / (cumulative_sum * (1 - cumulative_sum))
